In [1]:
import pandas as pd
import numpy as np
import os
import openai
import dotenv
from openai import OpenAI
import sys
import pandas as pd
import json
from dotenv import load_dotenv
import re

In [2]:
# nervis o angoixa
# Carreguem les notícies de desembre
df = pd.read_csv('..//datasets//2024_desembre_dataset.csv', low_memory=False)

In [4]:
# Carreguem la API KEY
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_api_key

# Models disponibles
models = openai.models.list()
model_list = [model.id for model in models.data]
print(model_list)

['gpt-4.5-preview', 'omni-moderation-2024-09-26', 'gpt-4.5-preview-2025-02-27', 'gpt-4o-mini-audio-preview-2024-12-17', 'dall-e-3', 'dall-e-2', 'gpt-4o-audio-preview-2024-10-01', 'gpt-4o-audio-preview', 'gpt-4o-mini-realtime-preview-2024-12-17', 'gpt-4o-mini-realtime-preview', 'o1-mini-2024-09-12', 'o1-preview-2024-09-12', 'o1-mini', 'o1-preview', 'gpt-4o-mini-audio-preview', 'whisper-1', 'gpt-4-turbo', 'gpt-4o-realtime-preview-2024-10-01', 'gpt-4', 'babbage-002', 'gpt-4-turbo-preview', 'tts-1-hd-1106', 'gpt-4o-audio-preview-2024-12-17', 'gpt4o-dream-ev3', 'tts-1-hd', 'gpt-4o-mini-2024-07-18', 'o3-mini', 'gpt-4-0125-preview', 'o3-mini-2025-01-31', 'tts-1', 'tts-1-1106', 'gpt-4o-2024-11-20', 'gpt-4-turbo-2024-04-09', 'davinci-002', 'gpt-3.5-turbo-1106', 'gpt-4o-2024-08-06', 'gpt-4o-mini', 'gpt-4o-2024-05-13', 'o1', 'gpt-3.5-turbo-instruct', 'o1-2024-12-17', 'chatgpt-4o-latest', 'gpt-4o', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo-0125', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-3.5-

In [5]:
# Estudiarem com de positives o negatives són les notícies
template = '''
El següent és un article de notícies. Llegeix-lo i realitza la tasca que segueix. Respon amb un objecte JSON de parells clau-valor en llengua catalana.

####################


{article}


####################


Tasca: Determina si l'article pot causar nervis o angoixa al lector.


1. Instrucció: Determina quin és el sentiment que pot predominar a una persona que llegeix la notícia d'entre "nerviós", "angoixat" o "altre".
   Key: "efecte"
   Value: Has de retornar un dels següents valors: "nerviós", "angoixat" o "altre".


2. Instruction: Retorna un paràgraf de justificació de la teva resposta. Justifica per què creus que l'article pot causar nervis o angoixa. Si la resposta és "altre", explica per què no causa nervis ni angoixa.
   Key: "justificacio"
   Value: Paràgraf de text.

No retornis res més que l'objecte JSON de parells clau-valor com a sortida.
'''

In [6]:
client = OpenAI()


def generate_api_response_model_4o(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "developer", "content": "Contestaràs tot en català."},
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

In [7]:
def generate_prompt(article, template):
    article_cos = article['cos']
    article_cos = str(article_cos)
    prompt = template.replace("{article}", article_cos)
    return prompt

In [8]:
def repair_json(json_string):
    # Try to extract JSON (objects or arrays)
    json_match = re.search(r"(\{.*\}|\[.*\])", json_string, re.DOTALL)
    if json_match:
        json_content = json_match.group(0)
        try:
            parsed_json = json.loads(json_content)
            print("Valid JSON after extraction:", parsed_json)
            return parsed_json
        except json.JSONDecodeError as e:
            raise ValueError("No JSON object found in response")
    else:
        #throw error
        raise ValueError("No JSON object found in response")


In [50]:
row = df.sample()
row['cos']

1529    ACN Sabadell - L'Àrea TEDAX-NRBQ dels Mossos d'Esquadra ha incorporat dos nous robots d'última generació especialitzats en la desactivació d'artefactes explosius. Amb un cost d'uns 500.00 euros cadascun finançats amb fons europeus, són més lleugers i polivalents que els actuals, que ja tenen més de 15 anys. Eduard Bosch, cap de l'àrea TEDAX, ha destacat que els nous robots disposen de geometria variable, el que els permet adaptar la seva amplada a les necessitats de cada espai i d'uns sensors que detecten la contaminació en àmbit nuclear, radioactiu, biològic o químic. També són més fàcils de maniobrar, el que permetrà als especialistes treballar amb més seguretat i posant el focus en l'amenaça.Durant l'acte de presentació, Miquel Hueso, cap de la Comissaria General d'Intervenció, ha destacat que la incorporació dels dos nous robots representa una aposta decidida per la tecnificació del cos i suposa un canvi de paradigma en els models de treball utilitzats fins ara per fer fron

In [51]:
row['titol']

1529    Els Mossos d'Esquadra incorporen dos robots d'última generació especialitzats en la desactivació d'artefactes
Name: titol, dtype: object

In [52]:
prompt = generate_prompt(row, template)
response = generate_api_response_model_4o(prompt)


In [53]:
print(response)

```json
{
  "efecte": "altre",
  "justificacio": "L'article presenta una millora tecnològica en la capacitat dels Mossos d'Esquadra per gestionar situacions perilloses mitjançant nous robots. Malgrat que es tracta de temes relacionats amb artefactes explosius i amenaces de crim organitzat, el contingut se centra en les habilitats dels robots i com augmenten la seguretat dels agents i del públic. No s'exposa cap incident recent que pugui causar alarma immediata. Així doncs, la informació pot ser percebuda més com una notícia informativa i de progrés, en lloc de causar nervis o angoixa."
}
```


In [ ]:
df_sample = df.sample(60)


60

In [57]:
results = {}
i = 0

for _, row in df_sample.iterrows():
    print(f"Processing row {i} of 60")
    i +=1
    prompt = generate_prompt(row, template)
    response = generate_api_response_model_4o(prompt)
    id = row['id']

    # Check if the response is valid JSON
    try:
        response_json = json.loads(response)  # Try to parse the response as JSON
        results[row['id']] = response_json   # Store the parsed JSON under the id
    except json.JSONDecodeError:
        try:
            response_json = repair_json(response)
            results[row['id']] = response_json
        except ValueError as e:
            print("FATAL ERROR: Could not extract JSON from response")
            results[row['id']] = {"error": "Invalid JSON response"}
            break

# Convert the results dictionary to a JSON string
final_json = json.dumps(results, indent=4)
print(final_json)

Processing row 0 of 60
Valid JSON after extraction: {'efecte': 'altre', 'justificacio': "L'article proporciona informació sobre els rècords històrics en l'ús del transport públic a Barcelona i Catalunya, destacant un augment en el nombre de viatgers tant en bus com en metro. Aquest tipus de notícies, centrades en estadístiques i dades positives, no tenen implicacions negatives evidents que podrien causar nervis o angoixa al lector. A més, no es mencionen problemes que afectin directament la seguretat o la qualitat del servei, per tant, el sentiment predominant probablement sigui de neutralitat o d'interès, en lloc de nervis o angoixa."}
Processing row 1 of 60
Valid JSON after extraction: {'efecte': 'altre', 'justificacio': "L'article tracta sobre les accions que es duen a terme per gestionar la sobrepoblació de coloms a Barcelona. Tot i que pot ser una preocupació per a algunes persones, especialment aquelles afectades directament per la brutícia o les plagues associades als coloms, l'

In [58]:
# Save to a file
with open('nervis.json', 'w') as json_file:
    json_file.write(final_json)

print("JSON file has been saved as 'results.json'.")

JSON file has been saved as 'results.json'.


In [ ]:
 ### pagina 23
# Estudiarem com de positives o negatives són les notícies
template_evitacio = '''
El següent text és una notícia. Llegeix-la i realitza la tasca que segueix. Respon amb un objecte JSON de parells clau-valor en llengua catalana.

####################


{article}


####################

Context: ets un assistent periodístic, que ha de determinar si l'article pot causar una resposta emocional negativa que pot portar als lectors a evitar les notícies. Per tal que una notícia no generi una resposta negativa es poden realitzar tres intervencions:

1. Incorporació del periodisme constructiu, que no només informa sobre problemes, sinó que també presenta solucions i perspectives d’esperança, ajudant a reduir la sensació de frustració o impotència entre els consumidors de notícies
2. Producció de contingut visual i narratiu atractiu, utilitzant històries humanes i exemples de canvi social positiu per captar l’atenció i generar un vincle positiu amb l’audiència
3. Dissenyar continguts que equilibrin les notícies sobre problemàtiques amb narratives que destaquin iniciatives positives o impactes socials dins del mateix relat informatiu. Això es podria aconseguir incorporant exemples d’esforços de millora, històries d’èxit o avanços significatius en àmbits relacionats amb la temàtica tractada. L’objectiu és oferir una visió més completa i matisada que ajudi a mantenir la motivació per consumir informació sense renunciar a la complexitat dels temes tractats. Aquest enfocament permet evitar una segregació artificial de les “bones notícies” integrant-les de manera orgànica dins del relat informatiu, i reforça un periodisme que inspira acció i reflexió, en lloc de generar frustració o desconnexió

Tasca: Determina si l'article pot causar una resposta emocional negativa que pot portar als lectors a evitar les notícies, o si es poden realitzar intervencions per evitar aquesta resposta.



'''

In [15]:
# set full widht pandas
pd.set_option("display.max_colwidth", None)  # Show full content of text columns

In [49]:
row = df.sample()
row['titol']

1468    S'aprova tramitar la llei per regular els lloguers de temporada, ara sí amb el vistiplau de Junts
Name: titol, dtype: object

In [50]:
prompt = generate_prompt(row, template_evitacio)
response = generate_api_response_model_4o(prompt)


In [51]:
print(response)

```json
{
  "potCausarRespostaNegativa": true,
  "intervencionsRecomanades": [
    {
      "tipus": "Periodisme Constructiu",
      "descripció": "Integrar informació sobre iniciatives i solucions de regulació de lloguers que han tingut èxit en altres regions o països per mostrar possibles camins cap a la millora de la situació dels lloguers temporada."
    },
    {
      "tipus": "Contingut Visual i Narratiu Atractiu",
      "descripció": "Incloure històries humanes de persones o famílies que s'han beneficiat de mesures legislatives similars o d'algun suport social per millorar la seva situació habitacional."
    },
    {
      "tipus": "Equilibri en Continguts",
      "descripció": "Incorporar exemples d'esforços socials o governamentals que busquen millorar l'accés a l'habitatge. Això pot incloure programes de promoció d'habitatge públic que hagin tingut impactes positius a curt termini dins de Catalunya o Espanya."
    }
  ]
}
```


In [ ]:
1. Instrucció: Determina


1. Instrucció: Determina quin és el sentiment que pot predominar a una persona que llegeix la notícia d'entre "nerviós", "angoixat" o "altre".
   Key: "efecte"
   Value: Has de retornar un dels següents valors: "nerviós", "angoixat" o "altre".


2. Instruction: Retorna un paràgraf de justificació de la teva resposta. Justifica per què creus que l'article pot causar nervis o angoixa. Si la resposta és "altre", explica per què no causa nervis ni angoixa.
   Key: "justificacio"
   Value: Paràgraf de text.